
### Libs

In [0]:
from pyspark.sql import SparkSession
import pandas as pd  
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import Row
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.appName("nacionalPauper").getOrCreate()

### Leitura dos dados

In [0]:
df_facebook = pd.read_csv("seu/caminho/scrapper.csv")
df_geral = pd.read_csv("seu/caminho/lista_jogadores.csv")

# df_facebook = gsheet.get_dataset(spreadsheet_id = '1IuOcyr0P5wGclH3RoZzl6qyjbKmD3NcgCj9s57O-T8A', range = 'Scrapper decklist!A:N')
# df_geral = gsheet.get_dataset(spreadsheet_id = '1IuOcyr0P5wGclH3RoZzl6qyjbKmD3NcgCj9s57O-T8A', range = 'Lista de jogadores!A:H')

df_facebook.display()
df_geral.display()

[INFO][2023-11-20 20:37:28,743][gsheet.py ][ 96][MainThread] - Schema length: 5
[INFO][2023-11-20 20:37:28,810][gsheet.py ][ 74][MainThread] - Read 248 rows
[INFO][2023-11-20 20:37:28,810][gsheet.py ][ 75][MainThread] - Schema:
root
-- id: string (nullable = true)
-- name: string (nullable = true)
-- content: string (nullable = true)
-- posted_on: string (nullable = true)
-- post_url: string (nullable = true)

[INFO][2023-11-20 20:37:28,811][gsheet.py ][ 76][MainThread] - None
[INFO][2023-11-20 20:37:30,189][gsheet.py ][ 96][MainThread] - Schema length: 6
[INFO][2023-11-20 20:37:30,255][gsheet.py ][ 74][MainThread] - Read 267 rows
[INFO][2023-11-20 20:37:30,255][gsheet.py ][ 75][MainThread] - Schema:
root
-- nome: string (nullable = true)
-- data: string (nullable = true)
-- loja: string (nullable = true)
-- cidade: string (nullable = true)
-- estado: string (nullable = true)
-- deck: string (nullable = true)

[INFO][2023-11-20 20:37:30,257][gsheet.py ][ 76][MainThread] - None

id,name,content,posted_on,post_url
pfbid027QF6qtPDuhNZAkbhpD62qqvpXYmt7FQRpgEqD4j385fZAVwVjFCkjCatqqiEw1ASl,Nacional Pauper,"INSCRIÇÕES PARA A FINAL DO NACIONAL PAUPER 2023Olá pessoal,Já estão abertas as inscrições para o Nacional Pauper 2023Apenas jogadores classificados poderão jogar.As inscrições devem ser pagas até o dia 26/11/2023, após essa data não serão mais aceitos pagamentos. Lembrando que quem não pagou não poderá jogar a final.Envie um e-mail para nacionalpauper@outlook.com , informando seu nome e a loja na qual conquistou a vaga para receber os dados e instruções de pagamento.Até mais!",2023-11-15 17:12:14,https://www.facebook.com/NacionalPauper/posts/pfbid027QF6qtPDuhNZAkbhpD62qqvpXYmt7FQRpgEqD4j385fZAVwVjFCkjCatqqiEw1ASl?__cft__[0]=AZWCsg1a98mwwktqmNip-AUzCw60GW5UgYCZfE5YsBAc_5AG0j_yJllFqKMqYVcttogrwou6Pr4T34f_i6ldnb_Yf-yUAualNv7qQOX8t9bKeb5-C3Wf2ghJ6LJ-OrT6ivp7gycdOXpnCbnHcGRMea3hJ8S4EJWfgSXXB1cebpnHcuuQMLAUDIbhs3HlRECZ8KghLkfe8ke615o9RQgmg4PZ&__tn__=%2CO%2CP-R
pfbid02ZJDAZBFfmywppTakbTyjYVDrrNC8bZT8V8nDL634Xip2UUkGXMpPX5LBc7Y9YQs3l,Nacional Pauper,Deck list do jogador classificado pela Meruru Liberdade - São Paulo - SPFábio DiasURMain Deck:2 Fathom Seer4 Phyrexian Walker11 Island4 Meeting of Minds1 Deceive the Messenger3 Ornithopter4 Molten Tributary4 Augur of Bolas4 Lorien Revealed3 Gigadrowse4 Hard Evidence4 Fallaji Archaeologist1 Shared Discovery4 Startle3 Springleaf Drum1 Keep Watch3 Rites of InitiationSideboard:2 Counterspell2 Relic of Progenitus3 Hydroblast3 Enchantment Alteration1 End the Festivities1 Rites of Initiation1 Skyshaper1 Callous Dismissal1 Snapback--------------------------------------------------------------------------Gostaria de participar ou organizar nossos classificatórios?Contato: nacionalpauper@outlook.comNacional Pauper - Não fique de fora!!!,2023-11-15 17:12:20,https://www.facebook.com/NacionalPauper/posts/pfbid02ZJDAZBFfmywppTakbTyjYVDrrNC8bZT8V8nDL634Xip2UUkGXMpPX5LBc7Y9YQs3l?__cft__[0]=AZWfGEyjwVr4U8oUvs3qv1Dl6Kps3VP-5fozcSeDvSfg6eHimQjz46e105YYqL9rTxsM89DZJQXSiOocPoM-l_RnSw4ybEdfsmZ_9f6QwVCoXYNzgZWvzohvnSSfMNB_t-fQKJEt7TAPQNnT85Nh0Vx7_egfcozoVfcdL41KkD-g_u_0_DHXBXK0IdcC7UvViuoMU9ShGU2harnOw9WoNoBW&__tn__=%2CO%2CP-R
pfbid06Zs7jKwE5HjdZgr812BPXFjZsRkrXiyr3iUg9WtYmUEez9i8XefgguaTxnMQgPcxl,Nacional Pauper,Deck list do jogador que recebeu bye 1 pela Meruru Liberdade - São Paulo - SPArtur SibutaUB FaeriesMain Deck:4 Contaminated Aquifer10 Island2 Swamp1 Bojuka Bog1 Maestros Theater1 Dispel2 Preordain3 Brainstorm3 Faerie Seer4 Augur of Bolas1 Chainer's Edict4 Counterspell1 Echoing Decay2 Cast Down1 Agony Warp4 Spellstutter Sprite1 Suffocating Fumes2 Thorn of the Black Rose4 Snuff Out3 Ninja of the Deep Hours4 Lorien Revealed2 Gurmag AnglerSideboard:1 Dispel3 Blue Elemental Blast3 Hydroblast2 Chainer's Edict1 Agony Warp1 Unexpected Fangs1 Suffocating Fumes1 Arms of Hadar1 Murmuring Mystic1 Okiba-Gang Shinobi--------------------------------------------------------------------------Gostaria de participar ou organizar nossos classificatórios?Contato: nacionalpauper@outlook.comNacional Pauper - Não fique de fora!!!,2023-11-15 17:12:20,https://www.facebook.com/NacionalPauper/posts/pfbid06Zs7jKwE5HjdZgr812BPXFjZsRkrXiyr3iUg9WtYmUEez9i8XefgguaTxnMQgPcxl?__cft__[0]=AZWocixg5SczuCSFHGfWOREfvUv2JZa1F_4kGy__uU7w9Ysmv9HPdAsrh5dxgOlCZ_YrJ9R3DNKIDj9h-lluAL59rtWRbHVsPAbl9cDoW2T1oBiV5XzNmHyiN8YOIkI76KsvSerXFWSPJa0r-XK5feYOMpeGk0kUl8acVv8-zEHWQB5PpHUtHSnmu6sGB6YgiooH_zBzorPpj9RSBY0sot0_&__tn__=%2CO%2CP-R
pfbid0LvoxP5JJkPoDbkuH6AjSxt39UnZRbZoAwZPNws32z66tqMmGs8pYNAvstFqB4uhYl,Nacional Pauper,Deck list do jogador que recebeu bye 1 pela Meruru Liberdade - São Paulo - SPPedro SanchesBGMain Deck:2 Haunted Mire1 Witch's Cottage10 Swamp1 Golgari Rot Farm4 Khalni Garden3 Crypt Rats2 Thorn of the Black Rose4 Avenging Hunter2 Troll of Khazad-dûm2 Candy Trail1 Blood Fountain3 Lembas2 Ichor Wellspring1 Duress4 Chainer's Edict3 Defile2 Reckoner's Bargain4 Deadly Dispute4 Cast Down1 Suffocating Fumes2 Snuff Out2 Spi

nome,data,loja,cidade,estado,deck
Adelar Tiemann Junior,10/10,Lumus Jogos e Colecionáveis,Barueri,SP,Goblins
Adriano Camilotti,07/10,Manabolt,Limeira,SP,BW
Adriano de Freitas,24/09,Imperium TCG,Espírito Santo do Pinhal,SP,UB Fadas
Adrien Venancio,15/11,Cards Hall,Sorocaba,SP,RW
Akio Okisasaki Ferrari,08/05,Vila Celta,Curitiba,PR,Dinrova Tron
Alaor Junior,08/10,Imperium TCG,Espírito Santo do Pinhal,SP,Mono Black
Alexandre Brianti,29/10,Arcana MTG,Itapira,SP,Burn
Alexandre Takano,15/11,Forjamundos,Campinas,SP,Mono Black
Alex Zanini,26/09,Alcatéia Games,Mogi-Mirim,SP,Golgari Control
Alfredo Obregon,12/08,Forjamundos,Campinas,SP,Orzhov Ephemerate



### Funções

In [0]:
def add_comma_between_letter_and_digit(text):
    return re.sub(r'([A-Za-z])(\d)', r'\1 \2', text)

add_comma_udf = udf(add_comma_between_letter_and_digit, StringType())

### ETL

In [0]:
regex_main_deck = r"Main Deck:(.*?)(?=Sideboard|$)"
regex_sideboard = r"Sideboard:(.*?)(?=--|$)"
regex_player = r"\b(SP|RJ|MG|MT|PR|SC|RS)\w*.*?\d"

df_facebook = (df_facebook.withColumn("main_deck_data", F.regexp_extract(F.col('content'), regex_main_deck, 1))
       .withColumn("sideboard_data", F.regexp_extract(F.col('content'), regex_sideboard, 1))
       .withColumn("player_name_data", F.regexp_extract(F.col('content'), regex_player, 0))
       )

df_facebook = (df_facebook.withColumn("main_deck_adjusted", add_comma_udf(F.col('main_deck_data')))
        .withColumn("sideboard_adjusted", add_comma_udf(F.col('sideboard_data'))))


# df_facebook.display()

In [0]:
lista_nomes = df_geral.select('nome').collect()
lista_decklists = df_facebook.select('player_name_data').collect()

resultado_nome = []
resultado_decklist_chave =[]

for decklist in lista_decklists:
  for nome in lista_nomes:
    match = re.search(nome.nome, decklist.player_name_data)
    if match:
      nome_jogador = match.group(0).strip()
      resultado_nome.append(nome_jogador)
      resultado_decklist_chave.append(decklist.player_name_data)

rdd = spark.sparkContext.parallelize([Row(nome_jogador=value1, chave=value2) for value1, value2 in zip(resultado_nome, resultado_decklist_chave)])
df_chave = spark.createDataFrame(rdd)

# df_chave.display()

### Join

In [0]:
final_df = (df_geral.join(df_chave, df_geral["nome"]==df_chave['nome_jogador'], 'left')
                    .join(df_facebook, df_facebook["player_name_data"] == df_chave['chave'], 'left')
                    )
final_df.display()

nome,data,loja,cidade,estado,deck,nome_jogador,chave,id,name,content,posted_on,post_url,main_deck_data,sideboard_data,player_name_data,main_deck_adjusted,sideboard_adjusted
Ana Clara,26/07,BWG,Campinas,SP,Mono Red,Ana Clara,SPAna Clara Mono RedMain Deck:4,pfbid02mXCgHfMSupcguBusjCJec2aHT3N7aDvAweGL8V1u7UK6xUGsrRKvWHHJZqbKreF2l,Nacional Pauper,Deck list da jogadora classificada pela BWG - Campinas - SPAna Clara Mono RedMain Deck:4 Great Furnace14 Mountain4 Voldaren Epicure4 Goblin Blast-Runner2 Goblin Bushwhacker4 Implement of Combustion4 Monastery Swiftspear4 Lightning Bolt3 Chain Lightning4 Kuldotha Rebirth4 Experimental Synthesizer4 Wrenn's Resolve2 Reckless Impulse2 Improvised Club1 FireblastSideboard:4 Raze2 End the Festivities1 Gorilla Shaman2 Flaring Pain4 Cast into the Fire2 Mine Collapse--------------------------------------------------------------------------Gostaria de participar ou organizar nossos classificatórios?Contato: nacionalpauper@outlook.comNacional Pauper - Não fique de fora!!!,2023-08-02 00:00:00,https://www.facebook.com/NacionalPauper/posts/pfbid02mXCgHfMSupcguBusjCJec2aHT3N7aDvAweGL8V1u7UK6xUGsrRKvWHHJZqbKreF2l?__cft__[0]=AZXDQcTgS6kKCF97VX4gTYsW5LKKym3xEl_kde6PwhsfKPibBsACRFqDhQd2VbmGm4nzmy98WcfIBLWU_TQ1348cQ4LaF8ojYkp2hwfoXnyrqkXgbn7u8wFZNaQX7NStpk-6us5O4_aE-O0D7hXOO2k9PLHJDa_iemX9Lpd7jy20hFNuinlyoUxLAKNCmGRN0QPSG3bcTEKHUvjvF0ljy3sF&__tn__=%2CO%2CP-R,4 Great Furnace14 Mountain4 Voldaren Epicure4 Goblin Blast-Runner2 Goblin Bushwhacker4 Implement of Combustion4 Monastery Swiftspear4 Lightning Bolt3 Chain Lightning4 Kuldotha Rebirth4 Experimental Synthesizer4 Wrenn's Resolve2 Reckless Impulse2 Improvised Club1 Fireblast,4 Raze2 End the Festivities1 Gorilla Shaman2 Flaring Pain4 Cast into the Fire2 Mine Collapse,SPAna Clara Mono RedMain Deck:4,4 Great Furnace 14 Mountain 4 Voldaren Epicure 4 Goblin Blast-Runner 2 Goblin Bushwhacker 4 Implement of Combustion 4 Monastery Swiftspear 4 Lightning Bolt 3 Chain Lightning 4 Kuldotha Rebirth 4 Experimental Synthesizer 4 Wrenn's Resolve 2 Reckless Impulse 2 Improvised Club 1 Fireblast,4 Raze 2 End the Festivities 1 Gorilla Shaman 2 Flaring Pain 4 Cast into the Fire 2 Mine Collapse
Aluysio Leal,24/06,Pauper Bauru,Bauru,SP,Caw Gates,Aluysio Leal,SPAluysio LealCaw GatesMain Deck:2,pfbid02HarMcgMJhEntuthTwZrtuv4oGTNuYtVdunMBa5yvNe8SmZ2Ug5g1DkjtkMe5hntzl,Nacional Pauper,Deck list do jogador classificado pela Pauper Bauru - Bauru - SPAluysio LealCaw GatesMain Deck:2 Dawnbringer Cleric4 Sacred Cat4 Squadron Hawk2 Guardian of the Guildpact4 The Modern Age1 Expedition Map4 Counterspell4 Brainstorm1 Dispel4 Prismatic Strands1 Fire // Ice1 Smash to Dust3 Preordain4 Journey to Nowhere1 Plains4 Sea Gate2 Heap Gate4 Citadel Gate4 Basilisk Gate3 Azorius Guildgate3 Island Sideboard:4 Blue Elemental Blast4 Dust to Dust4 Pyroblast3 Relic of Progenitus--------------------------------------------------------------------------Gostaria de participar ou organizar nossos classificatórios?Contato: nacionalpauper@outlook.comNacional Pauper - Não fique de fora!!!,2023-06-27 00:00:00,https://www.facebook.com/NacionalPauper/posts/pfbid02HarMcgMJhEntuthTwZrtuv4oGTNuYtVdunMBa5yvNe8SmZ2Ug5g1DkjtkMe5hntzl?__cft__[0]=AZX8PEpXw7vtN-KJ-BrVdGo_Zuh6QuLK8BY6XoPj27_lGZ0VpDxkQbgfnPJrGjNU20cr5UqKElUJR4pHORAHlkKOHVIa8NAT1p0qoJtf6h8goWacLcxrvWneOtzcHCn8prmSIGHymfvSAOWqbEBr7dyWGYsN3hv_c4KRR8PJoT6TZHVMvHgges9GiHIuuPfIwz-Z7abdri1_kwzBcux7z7Em&__tn__=%2CO%2CP-R,2 Dawnbringer Cleric4 Sacred Cat4 Squadron Hawk2 Guardian of the Guildpact4 The Modern Age1 Expedition Map4 Counterspell4 Brainstorm1 Dispel4 Prismatic Strands1 Fire // Ice1 Smash to Dust3 Preordain4 Journey to Nowhere1 Plains4 Sea Gate2 Heap Gate4 Citadel Gate4 Basilisk Gate3 Azorius Guildgate3 Island,4 Blue Elemental Blast4 Dust to Dust4 Pyroblast3 Relic of Progenitus,SPAluysio LealCaw GatesMain Deck:2,2 Dawnbringer Cleric 4 Sacred Cat 4 Squadron Hawk 2 Guardian of the Guildpact 4 The Modern Age 1 Expedition Map 4 Counterspell 4 Brainstorm 1 Dispel 4 P

In [0]:
columns = ['nome', 'data', 'loja', 'cidade', 'estado', 'deck', 'post_url', 'main_deck_adjusted', 'sideboard_adjusted']
final_df.select([F.lower(column).alias(column.lower()) for column in final_df.columns]).orderBy(F.col('nome').asc()).display()

nome,data,loja,cidade,estado,deck,nome_jogador,chave,id,name,content,posted_on,post_url,main_deck_data,sideboard_data,player_name_data,main_deck_adjusted,sideboard_adjusted
adelar tiemann junior,10/10,lumus jogos e colecionáveis,barueri,sp,goblins,null,null,null,null,null,null,null,null,null,null,null,null
adriano camilotti,07/10,manabolt,limeira,sp,bw,adriano camilotti,spadriano camilottibwmain deck:8,pfbid024mm4enjrfa8foru1m3h4oalggdxmrzobzkuvqbgjsyxfy11aftvxn7ndnqeybdykl,nacional pauper,deck list do jogador classificado pela manabolt - limeira - spadriano camilottibwmain deck:8 planicie5 pantano2 pantano de bojuka2 pescador celeste kor1 saqueador saco-de-carne3 basilica orzhov2 liquidar2 ratos da cripta4 inspetor de thraben2 espinho da rosa negra4 demover2 soberana vampira3 efemerar2 augurio dos mortos2 clerigo arauto da aurora4 companheira espirituosa4 supervisor inspirador2 paladino golias2 destruir o mal4 charco ensolaradosideboard:3 reliquia de progenitus3 edito de chainer3 portador do estandarte3 clerigo arashin3 exalacoes sufocantes--------------------------------------------------------------------------gostaria de participar ou organizar nossos classificatórios?contato: nacionalpauper@outlook.comnacional pauper - não fique de fora!!!,2023-10-12 00:00:00,https://www.facebook.com/nacionalpauper/posts/pfbid024mm4enjrfa8foru1m3h4oalggdxmrzobzkuvqbgjsyxfy11aftvxn7ndnqeybdykl?__cft__[0]=azur8f68pizb8-xcoqidrinspoqgct25nomymoqxy2yzrylo6jgk9zeu15lkas5oyf-xutssgqwkv9ctpaoitxtqt1jbcvtqf7ecmfuotpnpcqzhqnhogclsvobdb1yvnaz4tznns_sihpu-vgeob8pp_la75cbcqik77vzv8sz-pxgxj6uoim2fzpx_l4d7bu61kmi_tqlm_nrqvwwzrceh&__tn__=%2co%2cp-r,8 planicie5 pantano2 pantano de bojuka2 pescador celeste kor1 saqueador saco-de-carne3 basilica orzhov2 liquidar2 ratos da cripta4 inspetor de thraben2 espinho da rosa negra4 demover2 soberana vampira3 efemerar2 augurio dos mortos2 clerigo arauto da aurora4 companheira espirituosa4 supervisor inspirador2 paladino golias2 destruir o mal4 charco ensolarado,3 reliquia de progenitus3 edito de chainer3 portador do estandarte3 clerigo arashin3 exalacoes sufocantes,spadriano camilottibwmain deck:8,8 planicie 5 pantano 2 pantano de bojuka 2 pescador celeste kor 1 saqueador saco-de-carne 3 basilica orzhov 2 liquidar 2 ratos da cripta 4 inspetor de thraben 2 espinho da rosa negra 4 demover 2 soberana vampira 3 efemerar 2 augurio dos mortos 2 clerigo arauto da aurora 4 companheira espirituosa 4 supervisor inspirador 2 paladino golias 2 destruir o mal 4 charco ensolarado,3 reliquia de progenitus 3 edito de chainer 3 portador do estandarte 3 clerigo arashin 3 exalacoes sufocantes
adriano de freitas,24/09,imperium tcg,espírito santo do pinhal,sp,ub fadas,adriano de freitas,spadriano de freitasub fadasmain deck:4,pfbid02pnzemgqww2jc3yxvmwppdod6ew76bhhb85pwgv6nahmopod56ndqicfwyncbqpopl,nacional pauper,deck list do jogador classificado pela imperium tcg - espírito santo do pinhal - spadriano de freitasub fadasmain deck:4 terras em desenvolvimento2 vaga-pensador4 silfide magioclasta10 ilha da neve2 pantano da neve4 tempestade cerebral4 contramagica3 ninja das horas tardias2 degeneracao reverberante4 liquidar4 preordenar4 augure de nicol bolas2 pescador de grumag2 desolacao fumegante4 demover3 vidente fada2 tunel de gelosideboard:2 delir3 verdade reverberante4 coagir4 explosao elemental do azul2 geist da tempestade--------------------------------------------------------------------------gostaria de participar ou organizar nossos classificatórios?contato: nacionalpauper@outlook.comnacional pauper - não fique de fora!!!,2023-09-26 00:00:00,https://www.facebook.com/nacionalpauper/posts/pfbid02pnzemgqww2jc3yxvmwppdod6ew76bhhb85pwgv6nahmopod56ndqicfwyncbqpopl?__cft__[0]=azvj6daordqizzff-qpltv46jl5npqkhvpe52jujwqj_plwuls12tc7y4ainv44wa1g6chbc2jfyivvq1eaa5dn9nxkity6u_gniux-gqvkxbzfcq5bzkh3orc1i70qx6srtyxvucjgb7th5erib6fsi9y9zgf1hv6kybvqdp4xagiykn0babnjsutm1mdjsp0odfauhtvnnczz_pi8waho2&__tn__=%2co%2cp-r,4 terras em desenvolvimento2 vag